### Counting Words

Begin by enabling plots to be displayed in the Jupyter notebook and importing the pandas library. 

In [ ]:
%matplotlib inline

import pandas as pd
import seaborn as sns

The sample text for this section is from the UN General Debate Corpus. These were released on [kaggle](https://www.kaggle.com/unitednations/un-general-debates/home). Since CSV files aren't really well-designed to hold long text fields with line-breaks, quotation marks and commas (which might interpretated by a csv reader as the start of a new field), the addresses are in the json file format.

In [1]:
un_df = pd.read_json('files/un-general-debates.json')

NameError: name 'pd' is not defined

I use `info`, `describe`, and `sample` to get a sense of the data.

In [ ]:
un_df.info()

In [ ]:
un_df.describe()

In [ ]:
un_df.sample(5)

A specific row can be referenced using its index through `iloc`. Here's what the first 800 characters of the variable `sotu_text` look like from a random row.

In [ ]:
un_df.iloc[2305]

In [ ]:
print(un_df.iloc[235]['speech_text'][:280])

### Word Count

One the simplest things one might want to know about text is how long is it. This could be useful for basic descriptive questions, such as "Have UN speeches increased in length over time?" Word counts are also useful for normalized texts. A 200 word essay with 10 exclamation marks is quite different from a 20,000 word essay with 10 exclamation marks 

In [ ]:
from pdtext.tf import word_count

In [ ]:
sentence = "I wish to congratulate  you, Sir, on your election as President of the General  Assembly"

In [ ]:
word_count(sentence)

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Create a new sentence stored as a string. Count the words.  

</div>



Now that the function has been imported, it can applied to our text variable, `sotu_text` to create a new variable with the number of words in the address.

In [ ]:
un_df['speech_word_count'] = un_df['speech_text'].apply(word_count)

`describe`, `hist`, and `scatter` can provide some information on the new variable.

In [ ]:
un_df['speech_word_count'].describe()

In [ ]:
un_df['speech_word_count'].hist(bins = 25)

In [ ]:
sns.boxplot(x = "speech_year", 
            y = "speech_word_count",
            data =un_df)

The dataset can be subset to just a few informative columns and then the `sort_values` and `head`/`tail` methods can list the longest and shortest addresses.

In [ ]:
un_df.sort_values(by='speech_word_count').head(10)

In [ ]:
un_df.sort_values(by='speech_word_count', ascending=False).head(10)

In [ ]:
print(un_df.iloc[3433]['speech_text'][:200])

Cuba and Libyan leaders are the longwinded.

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Work as a team. Open a new notebook. In the `files` folder, there is a file called "trump_ge_speeches.json" which contains the general election speeches of Donald Trump.
What is the average (median) number of words in one of his speeches? In what city did he give his longest speech? On what date did he give his shortest speech. 
    
You can also access the file from the internet at <code>https://raw.githubusercontent.com/nealcaren/KULeuvenBigData/master/notebooks/data/trump_ge_speeches.json</code>



</div>


#### Word frequencies

```America is strong, America is proud, and America is free.```

into something like:

| america | and | free | is | proud | strong |
|:-------:|-----|------|----|-------|--------|
| 3       | 1   | 1    | 3  | 1     |    1   |



```America is strong, America is proud, and America is free.```


|word|freq|
|---|---|
|america|3|
|and|1|
|free|1|
|is|3|
|proud|1|
|strong|1|

Word frequencies are the backbone of almost all text analysis. From topic models to text classification, counting how often certain words occur is a critical step in quantifying texts. While it is certainly possible to compute word frequencies using your own functions, that is usually unnecessary, as many Python libraries can compute words frequencies. 


If you don't really care about which specific words are in a text, but are mostly using them for subsequent statistical analysis, you'll likely need them in the first, wide format. Here each text is a row and each word a variable. This is the modal format for how text is interpreted as numbers. 

Alternatively, if you want to know about which specific words, or types of words, are most common, you might favor the second, long approach. 

In either case, note that the ordering of words in the original sentence. With few notable exceptions, analysts take what is called a bag-of-words approach. This simplifying assumption, that word order doesn't really matter, has two things going for it. First, it is computationally much easier to assume that order of words in a sentence doesn't matter. Second, the results, as you will see, are often pretty solid. Fields like sociology, which is based on the idea that individuals are shaped by their surroundings, analyze individual survey data frequently to great success. Bag-of-words is like that. We know context matters, but modeling strategies that ignore this can still provide fairly good estimates.


In [ ]:
from pdtext.tf import make_wf_df

We start off with some sample sentences.

In [ ]:
sentences = ['Unequal exchange is ruining and impoverishing our peoples and must cease.',
            'Arms expenditures are irrational.', 
             'They must cease and the funds thus released must be used to finance development.']

In [ ]:
wf_df = make_wf_df(sentences)

In [ ]:
wf_df

We can create a variable in the dataframe with the original texts. Since `sentences` is a list of two items, and because we know the the order of `sentences` matches the order of `df`, it is fairly straightfoward to add the new column.

In [ ]:
wf_df['sentence'] = sentences

In [ ]:
wf_df

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Create a word frequency datframe based on the following sentences.
</div>

In [ ]:
seuss_sen = ['This one has a little star.', 
            'This one has a little car.', 
            'Say!',
            'What a lot of fish there are.']

If instead, we wanted the most frequent words, we could use the summary option.

In [ ]:
make_wf_df(sentences, summary = True)

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Create a word frequency datframe from our Suess sentences.
</div>

We can now create a new dataframe based on the UN texts

In [ ]:
un_wf = make_wf_df(un_df['speech_text'])

This rebuilt our vocabulary list, which is now much longer.

In [ ]:
len(un_wf.keys())

We can use slice to examine an arbitrary section of the vocabulary.

In [ ]:
un_wf.keys()[11510:11520]

As before, we next `transform` our corpus to a dense array using the `vectorizer`. Note that while we are building and transforming on the same data, this doesn't have to be the case.

In [ ]:
un_wf.head()

In [ ]:
un_wf.sum().sort_values(ascending=False)[:20]

We can create a new dataframe that merges our better word frequencies with the original UN dataframe. pandas `concat` takes a list of dataframes and merges them. The value of `axis=1` means that datasets should be merged sided by side -- each data set is providing new columns, not new rows. Critically, `concat` assumes that the cases are in the same order in each of the dataframes. If this is not the case (or you are not sure), and you are merging based on a the value of column, you should use `merge`.

In [ ]:
df_combined = pd.concat([un_df, un_wf], axis=1)

In [ ]:
df_combined.head()

If you wanted to stop here, either to pick up your analysis on a different day or to complete your work in a different program, you could export the combined dataframe. if you wanted to keep the address text in your file, I would recommend storing it as JSON to avoid potential issues with that long text field. `orient='records'` ensures that each item in the JSON will be a case. 

In [ ]:
df_combined.to_json('un_wf.json', orient='records')

Alternatively, if you didn't care about the text, or new you already had it saved in the original json file, you can drop the text field and export to a csv file. `axis=1` tells pandas that you want to drop a column, rather than a row.

In [ ]:
df_combined.drop('speech_text', axis=1).to_csv('sotu_wf.csv')

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Back in your team. What were the most common words used by Donald Trump in his speeches?

</div>

An alternate strategy to removing words entirely is construct weights that are based on how frequently a word occurs in a particular document compared to how frequently it appears in other documents. So a word like "of" would score low in every document if it commonly found and used in similar frequencies. A word like "America" might be used in every text, but some addresses might use it more frequently, it which case it would score high in just those instances. Finally, the highest scores would be associated with a word like "terrorism" might be not only rare, but also appear frequently in the few addresses that include it.  

The most common algorithm for this sort of word weight is called term-frequency/inverse document frequency, or TF-IDF. The numerator, term frequency, is how frequently a word occurs in a document divided by the number of words in the document. The denominator is the natural log of the fraction of the total number of documents divided by the number of documents with term in it. 

Returning to our two sample sentences:

Scikit-learn's `TfidfVectorizer` can be used to compute tf-idfs with identical syntax to the `CountVectorizer`.

![](http://dovgalecs.com/blog/wp-content/uploads/2012/03/img131.gif)

In [ ]:
def make_wf_df(text_column, summary=False, tfidf = False, **kwargs):
    if tfidf == True:
        vectorizer = TfidfVectorizer(**kwargs, token_pattern=r"(?u)\b\w+\b")
    else:
        vectorizer = CountVectorizer(**kwargs, token_pattern=r"(?u)\b\w+\b")

    wf = vectorizer.fit_transform(text_column)

    if type(text_column) == pd.core.series.Series:
        index_column = text_column.index
    else:
        index_column = range(0,len(text_column))

    word_freq_df = pd.DataFrame(
        wf.toarray(), columns=vectorizer.get_feature_names(), index = index_column
    )

    if summary == False:
        return word_freq_df

    summary_df = word_freq_df.sum().sort_values(ascending=False)
    return summary_df

In [ ]:
make_wf_df(sentences, tfidf = True)


As before, we can create apply the vectorizer to the SOTU addresses. I exclude stop words because, even with the idf adjustments, they still can overwhelm the results.

In [ ]:
un_tfidf_df = make_wf_df(un_df['speech_text'], tfidf = True)



As before, the original dataframe and the tf-idf dataframe can be combined with `concat`.

In [ ]:
un_df_combo2 = pd.concat([un_df, un_tfidf_df], axis=1)

In [ ]:
un_df_combo2.info()

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Group project again. Create a tf-idf dataframe from the trump speeches. Do it!

</div>



The `sum` method that was used for the word frequencies to find the most important words is less informative here. tf-idf is meant to identify important words within a text, rather than the most important words in an entire corpus, so the meaning of summing the values across multiple texts is not apparent.However, we can use them to identify the most uniquely informative words within a given text. 

The most useful way to analyze the most informative words (based on tf-idf) is to switch the dataframe from wide (where each case is a speech, to long, where each row is a word-case.  For our two sentence example, the result would look like this:



| sentence |	word|	value|
|-|-|-|
|0|	america|	0.535941
|0|	is	|0.535941
|0|	and|	0.376623
|0	|proud	|0.37662
|	0|	strong|	0.376623
|0|	free	|0.000000
|	1|	free|	0.704909
|	1|	america|	0.501549
|	1|	is|	0.501549
|	1|	and|	0.000000
|	1|	proud|	0.000000
|	1|	strong	|0.000000

The first sentence is about what "america" "is" while the second is about "free". 

In pandas, the `melt` method is used to convert a dataframe from wide to long. The first parameter is the name of the dataframe. `id_vars` is the variable or variables that will be used to identify the cases. In this case, that is `sotu_year`, which uniquely identifies each row. `value_vars` are the variables to be transposed. In this case, we only want the word variables, which are still stored in the `tfidf_vectorizer.get_feature_names()` list.

In [ ]:
long = pd.melt(un_tfidf_df)

In [ ]:
long.head()

In [ ]:
df_long = pd.melt(un_df_combo2, 
                  id_vars='sotu_year', 
                  value_vars=tfidf_vectorizer.get_feature_names())


In [ ]:
df_long.head()

The new dataframe is quite long.

In [ ]:
len(df_long)

Most of that, however, is zeros, so we can drop those cases.

In [ ]:
df_long = df_long[df_long['value']!= 0]

len(df_long)

For display purposes, it is also useful to set the word variable (which pandas renamed to `variable` as part of the `melt`) as the index.

In [ ]:
df_long.set_index('variable', inplace= True)
df_long.head()

Now, we can use `groupby` and `nlargest` to find the most important word, as measured by tf-idf value, for each year.

In [ ]:
df_long.groupby('sotu_year')['value'].nlargest(5)

I'm not that familiar with the early years of the republic, so 

In [ ]:
inaugaration_years = [1977, 1981, 1989, 1993, 2001, 2009, 2017]

df_inaugaration = df_long[ df_long['sotu_year'].isin(inaugaration_years) ]

In [ ]:
df_inaugaration.groupby('sotu_year')['value'].nlargest(5)

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Group project again. What were the most distinctive words used by Donald Trump in Detroit (2016-0903)

</div>




In [ ]:
df_inaugaration.groupby('sotu_year')['value'].nlargest(8).plot.barh(figsize=(5,20))

### Bonus Visualization - TSNE


The TSNE algorithm in scikit-learn follows the same logic of as the vectorizers:
1. Establish the parameters. In this case, we want to reduce or data to two dimensions.
2. Fit the model on the data.
3. Transform the data to the two dimensions. 




As noted above, one major use for term frequency and tf-idf arrays is as an intermediary step for subsequent analysis, such as text classification or topic models. These arrays are often very wide, meaning they have hundreds or thousands of columns, each associated with a distinct word. Analysis often use a second intermediary step in order to reduce the number of variables, using techniques such as principle component analysis (PCA). TSNE, t-distributed stochastic neighbor embedding, is an alternative algorithm for data reduction that is designed for visualization. It is usually used to reduce the number of variables to two, which become the x and y values for graphing. While the co-ordinates themselves have no meaning, similar cases are presented as clustered together.  

The TSNE algorithm in scikit-learn follows the same logic of as the vectorizers:
1. Establish the parameters. In this case, we want to reduce or data to two dimensions.
2. Fit the model on the data.
3. Transform the data to the two dimensions. 

For many scikit-learn functions, steps 2 (`fit`) and 3 (`transform`) can be combined with the `fit_transform` method, which we will use here.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2)

In [ ]:
coordinates = tsne.fit_transform(tfidf_df)

In [ ]:
coordinates[:10]

These coordinates can be stored in a dataframe.  

In [ ]:
tsne_df = pd.DataFrame(coordinates, 
                       columns=['tsne_y', 'tsne_x'])

In [ ]:
tsne_df.info()

Once in a dataframe, the tsne co-ordinates can be plotted. I use the `fig_size` option to ensure the result are square.  

In [ ]:
tsne_df.plot.scatter(y='tsne_y', 
                     x = 'tsne_x', 
                     title ='t-SNE Plot of Presidential Addresses',
                     figsize=(5,5)
                     ) 

Without year variables, this doesn't tell us much about the patterning of speeches. We can copy over the year variable from the original dataset.

In [ ]:
tsne_df['year'] = sotu_df['sotu_year']
tsne_df['year'].describe()

Ideally, the graph would allow you to select an individual point to find out the year, the president, and the text of the speech. This is possible with some graphing libraries, such as [bokeh](https://bokeh.pydata.org/en/latest/) or [plotly](https://plot.ly), but beyond the scope of this chapter.

Rather than labeling each point, we can bin them using the pandas `cut` method. For this case, I create ten different groups stored in the new variable `era`.

In [ ]:
tsne_df['era'] =  pd.cut(tsne_df['year'], 10)

[seaborn](https://seaborn.pydata.org) is a popular visualization library that makes the standard python plots prettier and easier to use. It also has the most convient method for coloring points in a scatterplot using a third variable, such as era.

By convention, seaborn is imported as `sns`.

In [ ]:
import seaborn as sns

`pairplot` can be used to produce a matrix of correlations, but we are only interested in our t-SNE coordinates. Note the syntax is slightly different here, as the x and y are just the column names and the dataframe name is set through the `data` parameter. 

In [ ]:
sns.pairplot(x_vars = 'tsne_x', 
             y_vars = 'tsne_y', 
             data   =  tsne_df, 
             hue    =  'era', 
             height=8)

This plot suggest that the language of presidential address is largely driven by temporal trends, rather than by party or region. A break seems to have occurred around the early 1920. A close inspection of this time period, through an examination of word frequencies or other quantitative or qualitative methods of text analysis, might suggest what the substance of the shift identified here.   

### Bonus: Bigrams

You might be interested not only in the frequency of certain words, but also word pairs with a distinct meaning, such as African American or United States. These word pairs are called bigrams, are more generally, ngrams.

If you have are familiar with your corpus and have a finite number of ngrams you want to handle, the best strategy is to preprocess the text yourself by writing a function that turns each of your ngrams into a single word.

In [ ]:
def my_ngrams(text):
    '''Replace ngrams in a string with unigrams'''
    text = text.replace("African American", "AfricanAmerican")
    text = text.replace("United States", "UnitedStates")
    text = text.replace("House of Representatives", "HouseofRepresentatives")
    return text

sotu_df['clean_sotu_text'] = sotu_df['sotu_text'].apply(my_ngrams)

The above function does a series of replaces, one for each ngram that you want to fix. Alternatively, you could create a list of the ngrams and then loop through them. This method scales up to larger lists much better and avoids lines of duplicate code. It also takes advantage of the fact that the replacement text is the original text with the spaces removed (or replacing `' '` with `''`). 

In [ ]:
ngrams = ['African American', 'United States', 'House of Representatives']

def my_ngrams2(text):
    '''Replace ngrams in a string with unigrams'''
    for ngram in ngrams:
        replacement = ngram.replace(' ','')
        text = text.replace(ngram, replacement)
    return text
sotu_df['clean_sotu_text'] = sotu_df['sotu_text'].apply(my_ngrams2)

Subsequent text analysis would be done on the new variable `clean_sotu_text` rather than the original, untransformed version.

Alternatively, you may want to identify ngrams in a more algorithmic way, either to identify candidates for your custom function or to use in your analysis. Both of scikit learns text vectorizers include an option to identify ngrams, `ngram_range`. It takes a pair of values, the first is the minimum value (where 1 is a single word) and the second is the maximum number. So `(1,2)` would construct a vocabulary of all unigrams and bigrams, while `(3,3)` would be only trigrams.

To identify the most common bigrams, we can rerun the analysis above, but this time setting with `ngram_range = (2,2)` as a parameter for the `CountVectorizer`.

In [ ]:
vectorizer = CountVectorizer(lowercase   = True,
                             ngram_range = (2,2),
                             stop_words  = 'english'
                             )

In [ ]:
bi_tf = vectorizer.fit_transform(sotu_df['sotu_text'])

In [ ]:
bi_tf_df = pd.DataFrame(bi_tf.todense(),
                        columns = vectorizer.get_feature_names())

In [ ]:
bi_tf_df.sum().sort_values(ascending=False)[:10]

Note that 'house representatives' likely was 'House of Representatives" in the original text, but the stop words removal happened prior to the ngram process. 

Replicating the process with both unigrams and ngrams:

In [ ]:
vectorizer = CountVectorizer(lowercase=True,
                             ngram_range = (1,2),
                             stop_words  = 'english'
                             )
bi_tf = vectorizer.fit_transform(sotu_df['sotu_text'])

bi_tf_df = pd.DataFrame(bi_tf.todense(),
                        columns = vectorizer.get_feature_names())

bi_tf_df.sum().sort_values(ascending=False)[:10]

Subsequent sections will discuss ways to decide how to algorithmically decide what are the best values for different vectorizer options, such as ngrams range or whether to include stop words or not.

### Bonus: Stemming

Should you count 'Americans' the same as "American"? Should "runs" and "run" be combined in your count? Having both singular and plural versions of the same noun or multiple variants of a verb can often clutter your analysis, particularly when looking at the most frequent words. In a large, diverse corpus, an article about "immigration" probably should be grouped with one about "immigrants", although this relationship might be obscured because of word variations. 

Two common strategies for handling this issues are stemming and lemmatization.  Stemming is a rule-based method to chop off the end of a word, such as removing "ing" at the end of a word so that "falling" becomes "fall" or the "d" for words that fit a pattern of constant followed by a "ed", such as "conflated" becomes "conflate" but "feed" does not become "fee". 

In contrast, lemmatization is a process of identifying the dictionary form of a word, or lemma. So while a stemmer might turn "communities" into "commun", the lemma is "community". Lemmatization works best with knowledge of the part of speech, as the lemma of the verb "saw" is "see", while the lemma of the noun "saw" is "saw".

Both methods have strengths and weaknesses. Stemming is fast and bins many related words together, but it can be too aggressive in chopping and the stems are often non-obviously interpretable. In contrast, lemmatization can be quite computationally intensive, especially when the text need to be part-of-speech tagged first. In practice, neither is commonly used, although they are useful tools to have knowledge of.

Both stemming and lemmatazation are available in Python through NLTK, the Natural Language Tool Kit. There are several variants of each available, but the most common are the `PorterStemmer` and the `WordNetLemmatizer`.

Note: If this is the first time you are using either, you will need to download auxilary files for them first.

```
import nltk
nltk.download('punkt')
nltk.download('wordnet')
```

In [ ]:
from nltk.stem.snowball import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
stemmer = PorterStemmer()

In [ ]:
stemmer.stem('falls')

In [ ]:
stemmer.stem('organizing')

In [ ]:
lemmer = WordNetLemmatizer()

In [ ]:
lemmer.lemmatize('falls')

In [ ]:
lemmer.lemmatize('organizing')

As you can see, both have similar syntax, although the lemmatizer also has the capacity to accept a part-of-speech tag. If defaults to assuming things are a noun. If the word is not found it its dictionary as a noun, it returns the original word. However, we can produce the verb lemma.

In [ ]:
lemmer.lemmatize('organizing', 'v')

Incorporating stemming or lemmatization in vectorizer requires passing a custom tokenizer. The standard tokenizer takes a text string, strips the punctuation and other one-character words, and returns a list of words. So to create a custom stemming tokenizer, we need a function that does the same but also stems each word.

In my `tokenize`, I use `word_tokenize` from nltk to split the sentences into words. The second line stems each word, excluding those tokens that are punctuation or stop words. This second second line uses list comprehension, a concise way to speed up list creation without using `for` loops. The one I employ below is equivalent to:

``` 
stems = []
for w in words:
    if keep(w) == True:
        s = stemmer.stem(w)
        stems.append(s)
```
\[Personally, I find it easier to read a `for` loop, but list comprehensions are fast and pithy.\]

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

stemmer = PorterStemmer()

stop_words = stopwords.words('english')

def keep(word):
    '''Test whether a token is a stop word or punctuation'''
    if word in punctuation:
        return False
    elif word in stop_words:
        return False
    return True
    
def tokenize_and_stem(text):
    '''Tokenize and stem a text returning a word list.'''
    words = word_tokenize(text)
    stems = [stemmer.stem(w) for w in words if keep(w) == True]
    stems 
    return stems

We can test that `tokenize` returns what we expect it to:

In [ ]:
tokenize_and_stem('America is strong because Americans work hardest under adversity.')

The `tokenize` function can be added as an option for a count vectorizer.

In [ ]:
stem_vectorizer = CountVectorizer(tokenizer = tokenize_and_stem)

In [ ]:
stem_tf = stem_vectorizer.fit_transform(sotu_df['sotu_text'])

stem_tf = pd.DataFrame(stem_tf.todense(),
                       columns = stem_vectorizer.get_feature_names())

In [ ]:
stem_tf.sum().sort_values(ascending=False)[:20]

Big winners here is "state" as the combination of "states" and "state", along with a few "stated". "Nation" combines "nation" and "nations" plus "national" and a few "nationality", but no "nationwide", which stems to "nationwid".